# **Teaching materials for AI beginners**  
**Original image recognition**  
Let's create an AI model with the prepared image.
Once you can do this, you will be able to create your own image classification model.  
We prepared 10 classes of 50 images for each class.  
'camera', 'chair', 'chandelier', 'crayfish', 'cup', 'dolphin', 'elephant', 'pizza', 'umbrella', 'watch'  
The more images you have, the higher the accuracy.  
You can also try changing to the collected image.
Just put the images in a folder.
It's very easy. Try it with your own image.

##1.Development environment preparation

###1-1.Connect to Google Drive.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


###1-2.Go to My Drive.

In [2]:
cd /content/gdrive/My\ Drive

/content/gdrive/My Drive


###1-3.Download the code from github.

In [3]:
!git clone https://github.com/ai-coodinator/Easy-and-free-AI-learning.git

Cloning into 'Easy-and-free-AI-learning'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 757 (delta 2), reused 0 (delta 0), pack-reused 747
Receiving objects: 100% (757/757), 7.51 MiB | 8.61 MiB/s, done.
Resolving deltas: 100% (86/86), done.
Checking out files: 100% (532/532), done.


##1-4.Change to the downloaded directory.

In [4]:
cd Easy-and-free-AI-learning/03_Original_Image_recognition/

/content/gdrive/My Drive/Easy-and-free-AI-learning/03_Original_Image_recognition


##1-5.Confirmation of GPU to use

In [5]:
!nvidia-smi

Mon Oct 26 11:54:51 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

##2.Data creation for learning  
This code transforms the prepared image data into a single training dataset.  
If you want to change or add training data, make the folder a class and place it in the Image folder.

In [6]:
from sklearn.model_selection import train_test_split
from PIL import Image
import os, glob
import numpy as np

# read image
root_dir = "./image/"
directory = os.listdir(root_dir)
categories = [f for f in directory if os.path.isdir(os.path.join(root_dir, f))]
print(categories)

num_classes = len(categories)
# resize
image_size = 224

# make image dataset
X = [] # image
Y = [] # label

# read image
for idx, cat in enumerate(categories):

    label = [0 for i in range(num_classes)]
    label[idx] = 1

    image_dir = root_dir + "/" + cat
    files = glob.glob(image_dir+"/*.jpg")
    for i, f in enumerate(files):
        img = Image.open(f)
        img = img.convert("RGB")
        img = img.resize((image_size, image_size))
        data = np.asarray(img)
        X.append(data)
        Y.append(label)

X = np.array(X)
Y = np.array(Y)

# make npy
X_train, X_test, y_train, y_test = \
    train_test_split(X, Y, test_size=0.2)
xy = (X_train, X_test, y_train, y_test)
np.save("./image/" + "obj.npy", xy)

print("ok,", len(X_train), len(X_test))


['camera', 'chair', 'chandelier', 'crayfish', 'cup', 'dolphin', 'elephant', 'pizza', 'umbrella', 'watch']
ok, 400 100


##3.Training start code  
This code creates an AI model with a CNN model

In [7]:
import os

import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import numpy as np

save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'original_trained_model.h5'

# Specifying the image data folder
root_dir = "./image/"
directory = os.listdir(root_dir)
categories = [f for f in directory if os.path.isdir(os.path.join(root_dir, f))]
print(categories)

num_classes = len(categories)
batch_size = 32
epochs = 15

# Load model
x_train, x_test, y_train, y_test = np.load("./image/obj.npy", allow_pickle=True)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

# Normalize the data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('X_train shape:', x_train.shape)

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test),
          shuffle=True)

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Evaluate the model
score = model.evaluate(x_test, y_test)
print('loss=', score[0])
print('accuracy=', score[1])

['camera', 'chair', 'chandelier', 'crayfish', 'cup', 'dolphin', 'elephant', 'pizza', 'umbrella', 'watch']
x_train shape: (400, 224, 224, 3)
400 train samples
100 test samples
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 224, 224, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 222, 222, 32)      9248      
_________________________________________________________________
activation_1 (Activation)    (None, 222, 222, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
dropout (Drop

##4.predict start code
This code uses an AI model to classify image data.  
This code infers the image in the predict folder.

In [10]:
# coding:utf-8
import os
import re
import keras
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import array_to_img, img_to_array,load_img

from PIL import Image

root_dir = "./image/"
directory = os.listdir(root_dir)
categories = [f for f in directory if os.path.isdir(os.path.join(root_dir, f))]
print(categories)

num_classes = len(categories)

image_size = 224

def list_pictures(directory, ext='jpg|gif|png'):
    return [os.path.join(root, f)
            for root, _, files in os.walk(directory) for f in files
            if re.match(r'([\w]+\.(?:' + ext + '))', f.lower())]

def convertCIFER10Data(image):
    img = image.astype('float32')
    img /= 255
    c = np.zeros(image_size*image_size*3).reshape((1,image_size,image_size,3))
    c[0] = img
    return c

if __name__ == "__main__":

    model = load_model('./saved_models/original_trained_model.h5')
    for picture in list_pictures('./predict/'):

        image = Image.open(picture)
        image = image.resize((image_size, image_size))
        resize_frame = np.asarray(image)
        data = convertCIFER10Data(resize_frame)

        ret = model.predict(data, batch_size=1)

        print("----------------------------------------------")
        print("I think...")

        bestnum = 0.0
        bestclass = 0
        for n in range(num_classes):
            print("[{}] : {}%".format(categories[n], round(ret[0][n]*100,2)))
            if bestnum < ret[0][n]:
                bestnum = ret[0][n]
                bestclass = n

        print("probability : {}%".format( round(bestnum*100,2) ))
        print(picture,'→',"AI think this is a [{}].".format(categories[bestclass]))


['camera', 'chair', 'chandelier', 'crayfish', 'cup', 'dolphin', 'elephant', 'pizza', 'umbrella', 'watch']
----------------------------------------------
I think...
[camera] : 73.91%
[chair] : 0.74%
[chandelier] : 0.24%
[crayfish] : 2.64%
[cup] : 1.56%
[dolphin] : 1.87%
[elephant] : 7.39%
[pizza] : 0.64%
[umbrella] : 1.23%
[watch] : 9.78%
probability : 73.91%
./predict/camera.jpg → AI think this is a [camera].
----------------------------------------------
I think...
[camera] : 3.39%
[chair] : 7.47%
[chandelier] : 12.19%
[crayfish] : 12.63%
[cup] : 38.89%
[dolphin] : 2.96%
[elephant] : 6.62%
[pizza] : 2.4%
[umbrella] : 11.29%
[watch] : 2.17%
probability : 38.89%
./predict/chair.jpg → AI think this is a [cup].
----------------------------------------------
I think...
[camera] : 0.05%
[chair] : 14.2%
[chandelier] : 48.22%
[crayfish] : 4.85%
[cup] : 5.83%
[dolphin] : 0.12%
[elephant] : 0.57%
[pizza] : 16.08%
[umbrella] : 0.35%
[watch] : 9.72%
probability : 48.22%
./predict/chandelier.jpg →